![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b>
Advanced RAG mit LangGraph
</b></font> </br></p>

---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/GenAI.git#subdirectory=04_modul
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

# LangChain Version anzeigen
import langchain
print(f"\n✅ LangChain Version: {langchain.__version__}")

In [ ]:
#@title 🛠️ Installationen { display-mode: "form" }
install_packages([
    ('markitdown[all]', 'markitdown'),
    'langchain_chroma',
    'langchain_huggingface',
    'langgraph',
])

# 1 | Von Basic zu Advanced RAG
---

<p><font color='black' size='5'>
Motivation
</font></p>

Im Modul **M08** haben Sie die Grundlagen von RAG (Retrieval-Augmented Generation) kennengelernt:

```
User Query → Retrieve Documents → Generate Answer
```

**Das funktioniert gut, aber es gibt Probleme:**

❌ **Problem 1: Irrelevante Dokumente**
- Der Retriever findet manchmal unpassende Dokumente
- Diese verwirren das LLM und führen zu schlechten Antworten

❌ **Problem 2: Halluzinationen**
- Das LLM erfindet Fakten, die nicht in den Dokumenten stehen
- Keine Überprüfung der generierten Antwort

❌ **Problem 3: Unflexibel**
- Der Workflow ist statisch: Immer retrieve → generate
- Keine Anpassung an die Qualität der Zwischenergebnisse

**Lösung: Advanced RAG mit LangGraph!**

✅ **Self-RAG:** Bewertet gefundene Dokumente vor der Generierung
✅ **Corrective RAG:** Verbessert Queries bei schlechten Ergebnissen
✅ **Adaptive RAG:** Passt den Workflow dynamisch an

<p><font color='black' size='5'>
Übersicht der Patterns
</font></p>

| Pattern | Was macht es? | Wann verwenden? |
|---------|---------------|------------------|
| **Self-RAG** | Bewertet Relevanz der Dokumente | Wenn Retrieval-Qualität wichtig ist |
| **Corrective RAG** | Verbessert Query bei schlechten Retrievals | Wenn Nutzer-Queries ungenau sind |
| **Adaptive RAG** | Wählt beste Strategie basierend auf Query-Typ | Für vielseitige Anwendungen |

**In diesem Modul bauen wir:**
1. Self-RAG mit Document Grading
2. Corrective RAG mit Query Rewriting
3. Adaptive RAG mit Routing Logic

# 2 | Setup & Vorbereitung
---

In [ ]:
# Imports
from typing import TypedDict, List
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field

# LangGraph
from langgraph.graph import StateGraph, START, END

In [ ]:
# Modell und Embeddings
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

print("✅ LLM und Embeddings initialisiert")

<p><font color='black' size='5'>
Beispiel-Dokumente vorbereiten
</font></p>

Wir erstellen eine kleine Wissensbasis zu KI-Themen.

In [ ]:
# Beispiel-Dokumente
documents = [
    Document(
        page_content="LangGraph ist ein Framework zur Erstellung von zustandsbasierten, mehrstufigen KI-Anwendungen. Es verwendet Graphen mit Nodes und Edges.",
        metadata={"source": "langgraph_docs", "topic": "framework"}
    ),
    Document(
        page_content="Self-RAG bewertet die Relevanz von abgerufenen Dokumenten, bevor sie zur Generierung verwendet werden. Dies verbessert die Antwortqualität.",
        metadata={"source": "rag_paper", "topic": "rag"}
    ),
    Document(
        page_content="Corrective RAG erkennt schlechte Retrievals und formuliert die Query neu, um bessere Dokumente zu finden.",
        metadata={"source": "crag_paper", "topic": "rag"}
    ),
    Document(
        page_content="LangChain ist eine Bibliothek für die Entwicklung von Anwendungen mit Large Language Models. Es bietet Tools wie Chains, Agents und RAG.",
        metadata={"source": "langchain_docs", "topic": "framework"}
    ),
    Document(
        page_content="Embeddings sind numerische Repräsentationen von Text, die semantische Ähnlichkeit erfassen. Sie werden für Vektorsuche verwendet.",
        metadata={"source": "embeddings_guide", "topic": "embeddings"}
    ),
    Document(
        page_content="Halluzinationen treten auf, wenn LLMs Fakten erfinden, die nicht in den Eingabedaten vorhanden sind. Self-RAG hilft, dies zu reduzieren.",
        metadata={"source": "llm_hallucinations", "topic": "llm"}
    )
]

# Vektordatenbank erstellen
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="advanced_rag_demo"
)

# Retriever erstellen
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

print(f"✅ Vektordatenbank mit {len(documents)} Dokumenten erstellt")

# 3 | Self-RAG: Document Grading
---

<p><font color='black' size='5'>
Konzept
</font></p>

**Self-RAG fügt einen Bewertungsschritt hinzu:**

```
Query → Retrieve → [GRADE DOCS] → Generate (nur mit relevanten Docs)
```

**Warum wichtig?**
- Irrelevante Dokumente werden herausgefiltert
- Das LLM bekommt nur hochwertige Informationen
- Reduziert Halluzinationen und verbessert Genauigkeit

<p><font color='black' size='5'>
State Definition
</font></p>

In [ ]:
class SelfRAGState(TypedDict):
    """State für Self-RAG Workflow"""
    query: str                          # Nutzer-Anfrage
    documents: List[Document]           # Abgerufene Dokumente
    relevant_documents: List[Document]  # Gefilterte relevante Dokumente
    answer: str                         # Generierte Antwort
    relevance_scores: List[str]         # "yes"/"no" für jedes Dokument

<p><font color='black' size='5'>
Document Grader mit structured output
</font></p>

In [ ]:
# Pydantic-Modell für Bewertung
class GradeDocuments(BaseModel):
    """Bewertet ob ein Dokument relevant für die Query ist"""
    score: str = Field(
        description="'yes' wenn relevant, 'no' wenn nicht relevant"
    )

# Strukturiertes LLM für Grading
grader_llm = llm.with_structured_output(GradeDocuments)

# Grading Prompt
grade_prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein Experte für die Bewertung der Relevanz von Dokumenten."),
    ("human", """Bewertet, ob das folgende Dokument relevant für die Benutzeranfrage ist.

Dokument:
{document}

Benutzeranfrage:
{query}

Gib 'yes' zurück wenn das Dokument Informationen enthält, die zur Beantwortung der Anfrage relevant sind.
Gib 'no' zurück wenn das Dokument nicht relevant ist.""")
])

# Grading Chain
grader_chain = grade_prompt | grader_llm

print("✅ Document Grader erstellt")

<p><font color='black' size='5'>
Graph Nodes definieren
</font></p>

In [ ]:
def retrieve_node(state: SelfRAGState) -> SelfRAGState:
    """Ruft Dokumente ab"""
    query = state["query"]
    documents = retriever.invoke(query)
    print(f"🔍 Retrieved {len(documents)} Dokumente")
    return {"documents": documents}

def grade_documents_node(state: SelfRAGState) -> SelfRAGState:
    """Bewertet jedes Dokument auf Relevanz"""
    query = state["query"]
    documents = state["documents"]
    
    relevant_docs = []
    scores = []
    
    for doc in documents:
        # Dokument bewerten
        grade = grader_chain.invoke({
            "document": doc.page_content,
            "query": query
        })
        
        scores.append(grade.score)
        
        if grade.score == "yes":
            relevant_docs.append(doc)
            print(f"✅ Relevant: {doc.page_content[:50]}...")
        else:
            print(f"❌ Nicht relevant: {doc.page_content[:50]}...")
    
    print(f"\n📊 {len(relevant_docs)}/{len(documents)} Dokumente relevant")
    
    return {
        "relevant_documents": relevant_docs,
        "relevance_scores": scores
    }

def generate_node(state: SelfRAGState) -> SelfRAGState:
    """Generiert Antwort basierend auf relevanten Dokumenten"""
    query = state["query"]
    docs = state["relevant_documents"]
    
    # Kontext zusammenstellen
    context = "\n\n".join([doc.page_content for doc in docs])
    
    # RAG Prompt
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", "Du bist ein hilfreicher Assistent. Beantworte die Frage basierend auf dem Kontext."),
        ("human", """Kontext:
{context}

Frage: {query}

Antwort:""")
    ])
    
    # Generierung
    chain = rag_prompt | llm | StrOutputParser()
    answer = chain.invoke({"context": context, "query": query})
    
    print(f"\n🤖 Antwort generiert ({len(answer)} Zeichen)")
    
    return {"answer": answer}

print("✅ Nodes definiert")

<p><font color='black' size='5'>
Graph aufbauen
</font></p>

In [ ]:
# Self-RAG Workflow erstellen
self_rag_workflow = StateGraph(SelfRAGState)

# Nodes hinzufügen
self_rag_workflow.add_node("retrieve", retrieve_node)
self_rag_workflow.add_node("grade_documents", grade_documents_node)
self_rag_workflow.add_node("generate", generate_node)

# Edges definieren
self_rag_workflow.add_edge(START, "retrieve")
self_rag_workflow.add_edge("retrieve", "grade_documents")
self_rag_workflow.add_edge("grade_documents", "generate")
self_rag_workflow.add_edge("generate", END)

# Graph kompilieren
self_rag_app = self_rag_workflow.compile()

print("✅ Self-RAG Graph kompiliert")

<p><font color='black' size='5'>
Graph Visualisierung
</font></p>

In [ ]:
from IPython.display import Image, display

try:
    display(Image(self_rag_app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"⚠️ Graph-Visualisierung nicht verfügbar: {e}")

<p><font color='black' size='5'>
Self-RAG testen
</font></p>

In [ ]:
# Test Query
test_query = "Was ist Self-RAG und warum ist es wichtig?"

# Workflow ausführen
result = self_rag_app.invoke({
    "query": test_query
})

# Ergebnis anzeigen
mprint("## 🎯 Self-RAG Ergebnis")
mprint("---")
mprint(f"**Query:** {test_query}")
mprint(f"**Antwort:** {result['answer']}")

# 4 | Corrective RAG: Query Rewriting
---

<p><font color='black' size='5'>
Konzept
</font></p>

**Corrective RAG (CRAG) erkennt schlechte Retrievals und reagiert:**

```
Query → Retrieve → Grade
                      ↓
            Gut? → Generate
                      ↓
         Schlecht? → Rewrite Query → Retrieve wieder
```

**Warum wichtig?**
- Nutzer-Queries sind oft ungenau oder missverständlich
- Durch Umformulierung werden bessere Dokumente gefunden
- Erhöht Erfolgsrate drastisch

<p><font color='black' size='5'>
State Definition
</font></p>

In [ ]:
class CRAGState(TypedDict):
    """State für Corrective RAG"""
    original_query: str           # Ursprüngliche Query
    query: str                    # Aktuelle Query (kann umgeschrieben sein)
    documents: List[Document]
    relevant_documents: List[Document]
    answer: str
    retry_count: int              # Wie oft wurde umgeschrieben?
    has_relevant_docs: bool       # Flag für Routing

<p><font color='black' size='5'>
Query Rewriter
</font></p>

In [ ]:
# Query Rewriting Prompt
rewrite_prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein Experte für das Umformulieren von Suchanfragen."),
    ("human", """Die folgende Suchanfrage hat keine guten Ergebnisse geliefert.
Formuliere sie um, um bessere Dokumente zu finden.
Mache sie spezifischer und nutze Synonyme.

Ursprüngliche Query: {query}

Verbesserte Query:""")
])

# Rewriting Chain
rewriter_chain = rewrite_prompt | llm | StrOutputParser()

print("✅ Query Rewriter erstellt")

<p><font color='black' size='5'>
CRAG Nodes
</font></p>

In [ ]:
def crag_retrieve_node(state: CRAGState) -> CRAGState:
    """Retrieval für CRAG"""
    query = state["query"]
    documents = retriever.invoke(query)
    print(f"🔍 Retrieved {len(documents)} Dokumente für Query: '{query}'")
    return {"documents": documents}

def crag_grade_node(state: CRAGState) -> CRAGState:
    """Grading für CRAG mit Entscheidungslogik"""
    query = state["query"]
    documents = state["documents"]
    
    relevant_docs = []
    
    for doc in documents:
        grade = grader_chain.invoke({
            "document": doc.page_content,
            "query": query
        })
        
        if grade.score == "yes":
            relevant_docs.append(doc)
    
    # Entscheidung: Gibt es genug relevante Dokumente?
    has_relevant = len(relevant_docs) > 0
    
    if has_relevant:
        print(f"✅ {len(relevant_docs)} relevante Dokumente gefunden")
    else:
        print(f"❌ Keine relevanten Dokumente → Query wird umgeschrieben")
    
    return {
        "relevant_documents": relevant_docs,
        "has_relevant_docs": has_relevant
    }

def rewrite_query_node(state: CRAGState) -> CRAGState:
    """Schreibt Query um"""
    original = state["original_query"]
    current = state["query"]
    retry = state.get("retry_count", 0)
    
    # Query umschreiben
    new_query = rewriter_chain.invoke({"query": current})
    
    print(f"🔄 Query umgeschrieben (Versuch {retry + 1})")
    print(f"   Alt: '{current}'")
    print(f"   Neu: '{new_query}'")
    
    return {
        "query": new_query,
        "retry_count": retry + 1
    }

def crag_generate_node(state: CRAGState) -> CRAGState:
    """Generierung für CRAG"""
    query = state["original_query"]  # Verwende ursprüngliche Query für Antwort
    docs = state["relevant_documents"]
    
    context = "\n\n".join([doc.page_content for doc in docs])
    
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", "Du bist ein hilfreicher Assistent."),
        ("human", "Kontext:\n{context}\n\nFrage: {query}\n\nAntwort:")
    ])
    
    chain = rag_prompt | llm | StrOutputParser()
    answer = chain.invoke({"context": context, "query": query})
    
    return {"answer": answer}

print("✅ CRAG Nodes definiert")

<p><font color='black' size='5'>
CRAG Graph mit Conditional Routing
</font></p>

In [ ]:
# Routing Funktion
def route_after_grading(state: CRAGState) -> str:
    """Entscheidet: Generate oder Rewrite?"""
    retry_count = state.get("retry_count", 0)
    has_relevant = state["has_relevant_docs"]
    
    # Maximal 2 Rewrite-Versuche
    if has_relevant or retry_count >= 2:
        return "generate"
    else:
        return "rewrite_query"

# CRAG Workflow
crag_workflow = StateGraph(CRAGState)

# Nodes
crag_workflow.add_node("retrieve", crag_retrieve_node)
crag_workflow.add_node("grade", crag_grade_node)
crag_workflow.add_node("rewrite_query", rewrite_query_node)
crag_workflow.add_node("generate", crag_generate_node)

# Edges
crag_workflow.add_edge(START, "retrieve")
crag_workflow.add_edge("retrieve", "grade")

# Conditional Edge: Nach Grading
crag_workflow.add_conditional_edges(
    "grade",
    route_after_grading,
    {
        "generate": "generate",
        "rewrite_query": "rewrite_query"
    }
)

# Nach Rewrite wieder zu Retrieve
crag_workflow.add_edge("rewrite_query", "retrieve")
crag_workflow.add_edge("generate", END)

# Kompilieren
crag_app = crag_workflow.compile()

print("✅ CRAG Graph mit Conditional Routing kompiliert")

<p><font color='black' size='5'>
CRAG Graph Visualisierung
</font></p>

In [ ]:
try:
    display(Image(crag_app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"⚠️ Graph-Visualisierung nicht verfügbar: {e}")

<p><font color='black' size='5'>
CRAG testen
</font></p>

In [ ]:
# Test mit ungünstiger Query
test_query = "Wie funktioniert das Ding mit den Graphen?"

result = crag_app.invoke({
    "original_query": test_query,
    "query": test_query,
    "retry_count": 0
})

mprint("## 🎯 CRAG Ergebnis")
mprint("---")
mprint(f"**Ursprüngliche Query:** {result['original_query']}")
mprint(f"**Finale Query:** {result['query']}")
mprint(f"**Rewrite-Versuche:** {result.get('retry_count', 0)}")
mprint(f"**Antwort:** {result['answer']}")

# 5 | Vergleich: Basic vs. Advanced RAG
---

| Aspekt | Basic RAG (M08) | Self-RAG | Corrective RAG |
|--------|-----------------|----------|----------------|
| **Workflow** | Linear | Mit Grading | Mit Retry-Loop |
| **Dokument-Qualität** | Nicht geprüft | ✅ Geprüft | ✅ Geprüft |
| **Query-Verbesserung** | ❌ Nein | ❌ Nein | ✅ Ja |
| **Anpassungsfähig** | ❌ Statisch | ⚠️ Teilweise | ✅ Dynamisch |
| **Komplexität** | Niedrig | Mittel | Hoch |
| **Use Case** | Einfache Q&A | Qualitätskritisch | Unklare Queries |

**Empfehlung:**
- **Basic RAG:** Für Prototyping und klare Use Cases
- **Self-RAG:** Wenn Retrieval-Qualität wichtig ist
- **Corrective RAG:** Wenn Nutzer-Queries ungenau sind

# 6 | Best Practices & Tipps
---

<p><font color='black' size='5'>
Performance-Optimierung
</font></p>

**Grading ist teuer (LLM-Calls):**
- ✅ Cache Grading-Ergebnisse für identische Dokumente
- ✅ Paralleles Grading mit `asyncio` (für große Dokumentenmengen)
- ✅ Alternative: Lightweight Relevance Scorer (z.B. Cosine-Similarity-Threshold)

**Rewriting Limits:**
- ✅ Maximal 2-3 Rewrite-Versuche (sonst zu langsam)
- ✅ Fallback: Web-Search wenn kein Doc gefunden wird

**Monitoring:**
- ✅ Logge Rewrite-Rate (zu hoch → Retrieval-Problem)
- ✅ Tracke Relevance-Scores (zu niedrig → Daten-Problem)

<p><font color='black' size='5'>
Wann welches Pattern?
</font></p>

| Szenario | Empfohlenes Pattern |
|----------|---------------------|
| **Hohe Datenqualität, klare Queries** | Basic RAG |
| **Viele irrelevante Retrieval-Ergebnisse** | Self-RAG |
| **Nutzer mit ungenauen Fragen** | Corrective RAG |
| **Kritische Anwendung (Legal, Medical)** | Self-RAG + Hallucination Check |
| **Niedrige Latenz wichtig** | Basic RAG (schneller) |
| **Hohe Accuracy wichtig** | Corrective RAG (besser) |

# A | Aufgaben
---

<p><font color='black' size='5'>
Aufgabe 1: Hallucination Detection hinzufügen
</font></p>

Erweitern Sie Self-RAG um einen **Hallucination Check Node**:

1. Nach der Generierung: Prüfe ob die Antwort durch die Dokumente gestützt wird
2. Wenn Halluzinationen erkannt: Generiere Antwort neu mit stärkerem Prompt
3. Verwende `with_structured_output()` mit Schema: `{"is_grounded": bool, "explanation": str}`

<p><font color='black' size='5'>
Aufgabe 2: Hybrid Retrieval
</font></p>

Kombiniere Vektor-Suche mit Keyword-Suche:

1. Erstelle zwei Retriever: Einer mit Embeddings, einer mit BM25
2. Node der beide Retriever parallel aufruft
3. Fusion-Node der die Ergebnisse kombiniert (z.B. nach Score sortieren)

**Tipp:** Nutze `langchain_community.retrievers.BM25Retriever`

<p><font color='black' size='5'>
Aufgabe 3: Adaptive RAG
</font></p>

Implementiere **Adaptive RAG** das basierend auf Query-Typ entscheidet:

1. **Classifier Node:** Kategorisiere Query ("simple", "complex", "ambiguous")
2. **Routing:**
   - `simple` → Basic RAG
   - `complex` → Self-RAG
   - `ambiguous` → Corrective RAG
3. Teste mit verschiedenen Query-Typen

**Bonus:** Verwende `with_structured_output()` für Classification